# LLM Security - Prompt Injection
## Part 2 - Classification Using a Pre-trained LLM

In this notebook, we load the testing dataset and use a pre-trained large language model to spot malicious prompts without fine-tuning it.
> **INPUT:** the testing dataset loaded from Hugging Face library. <br>
> **OUTPUT:** the performance analysis of considered LLM.


### 1. INITIALIZATION

In [1]:
# Import necessary libraries and modules
import pandas as pd

In [2]:
# Set display options
pd.set_option('display.max_columns', None)

### 2. LOADING DATASET

Since we are using a pre-trained model without fine-tuning, there is no need to load the training data set.

In [3]:
# Initialize data set location and file name
data_file_path = "../data/raw/"
data_file_name_test = "test-00000-of-00001-701d16158af87368"
data_file_ext = ".parquet"

# Loading data set into a pandas DataFrame
data_test = pd.read_parquet(data_file_path + data_file_name_test + data_file_ext)

In [4]:
# Rename "text" column into "prompt"
data_test.rename(columns={"text":"prompt"}, inplace=True)

We already explored the dataset in the previous notebook, so we will directly proceed to the inference phase.

### 3. MODEL PREDICTION (RoBERTa)

In this experiment, we aim at using a pre-trained LLM without fine-tuning the model on the downstream task.

Since the prompts in the dataset combine different languages, we should rely on a multilingual model that is trained on text of the languages indicated in the existing prompts.

For this purpose, RoBERTa (Robustly optimized BERT approach), the enhanced version of BERT (Bidirectional Encoder Representations from Transformers) will be used.

RoBERTa is a transformer-based neural network model developed by Facebook AI, designed for natural language understanding tasks. Since our dataset is multilingual, we will use XLM-RoBERTa variant, a multilingual version of RoBERTa pre-trained on data containing 100 languages.

Given the original task of XLM-RoBERTa is to predict masked words, we have to tailor this goal towards predicting whether a prompt is an injection or not. For this reason, we will utilize the "zero-shot-classification" task provided by Hugging Face pipeline to achieve this goal.

This method can be referred to as transfer learning, which is using a model trained for one task to perform another one different from what it was originally trained for.

In [5]:
# Import pipeline functionality from the Hugging Face transformer's library
from transformers import pipeline

In [6]:
# Load the zero-shot classification pipeline with RoBERTa
classifier = pipeline(task="zero-shot-classification", model="xlm-roberta-large")

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


When you use the zero-shot classification pipeline, it essentially formulates the classification problem as a fill-in-the-blank task.

In other words, the zero-shot classification pipeline treats the problem of predicting a label for a given prompt as if we were filling in a blank in a sentence. The model is asked to predict the missing piece, which, in this case, is the label that best fits the context provided by the input prompt.

In [7]:
# Define a function to classify the prompt
def classify_prompt(prompt):
    
    # List of candidate labels (in this case, indicating whether the text is an injection or not)
    candidate_labels = ["Injection", "Normal"]
    
    # Perform zero-shot classification
    output = classifier(prompt, candidate_labels)
    
    # Return the results
    return 1 if output['labels'][0] == "Injection" else 0

In [8]:
# Apply classifier to the testing dataset
data_test["predicted_label"] = data_test["prompt"].apply(classify_prompt)

### 4. RESULT ANALYSIS

In [9]:
# Import performance metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Initialize a DataFrame to keep track of the models' performance
results = pd.DataFrame(columns=["accuracy", "precision", "recall", "f1 score"])

# Initialize actual and predicted labels
y_test = data_test["label"]
y_predict = data_test["predicted_label"]
    
# Calculate performance metrics
accuracy = accuracy_score(y_test, y_predict)
precision = precision_score(y_test, y_predict)
recall = recall_score(y_test, y_predict)
f1 = f1_score(y_test, y_predict)

# Store performance metrics
results.loc["Testing Data"] = [accuracy, precision, recall, f1]

In [10]:
# Check obtained results
results

,accuracy,precision,recall,f1 score
Testing Data,0.551724,0.551282,0.716667,0.623188


From the results above we can notice the following points:

- Accuracy of testing dataset is low, which indicates that the model was not able to capture the characteristics of injections in the dataset.
- The model's performance could benefit from fine-tuning, especially given the nature of the classification task involving injection prompts.
- The low performance is quite expected given XLM-RoBERTa model is mostly intended to be fine-tuned on the downstream task before being used for classification.
- The low performance can be also justified in terms of the period during which this model was trained, expectedly prior to when the concept of prompt injection has been formulated.

In summary, while the model shows moderate performance, there is room for enhancement, and fine-tuning on task-specific data is likely to yield improvements in precision, recall, and overall classification effectiveness.